In [1]:
###Python Packages###

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.colors import LogNorm
import math as mth
from decimal import *

import timeit
import Double_Null_utils as dnu
from scipy import optimize
import tables
from mpmath import *


In [2]:
###Initial Values###

M0=1.0
Q=0.95
Lambda=0.0000
scalarfield=False
vscalarfield=False



In [3]:
Tol=1
getcontext().prec=40
N=10
Nul=100
Nup=10**(40)
scal=1
umax=5.3
vmax=200
datatype=object


#ru0=5.0
#dr0v=.4
ru0=2.0
#dr0v=0.045
dr0v=-1/(4*-.375)*(1-2*M0/ru0+Q**2/ru0**2)
#dr0v=0.045

bdytype="stan"
scaltostan=False
solveaffine=False

Elist=[1]
#Elist=[1,2,4,8,16]

#uloc=[1/8,1/4,3/8,1/2,5/8]
#uloc=[1/5,2/5,3/5,4/5,9/10]
uloc=[8.2/10]

In [4]:
###Defining Needed Constants###

scalf=float(scal)
u0=0.0
v0=0.0
rv0=ru0
phiu0=0.0
phiv0=0.0

if M0==0.0:
    dv0=1/N    
else:
    dv0=M0/N
    


def fr(r,M0,Q,Lambda):
    return 1-2*M0/r+Q**2.0/r**2.0-Lambda*r**2.0/3.0

if Lambda>0 or Lambda<0:
    sol = optimize.root(fr,[0.687,1.3,50.0],args=(M0,Q,Lambda), method='hybr')
    rminus=sol.x[0]
    rplus=sol.x[1]
    rcosm=sol.x[2]
else:
    rplus=M0+(M0**2-Q**2)**(.5)
    rminus=M0-(M0**2-Q**2)**(.5)
    rcosm=0.0
    

wHD=True 

print("r+: "+str(rplus))
print("r-: "+str(rminus))
print("rc: "+str(rcosm))

###Predicting Run time###

###timer
start = timeit.default_timer()

Nu=Nul*umax*100#int(umax/du0)
Nv=int(vmax/dv0)
#print("Number of points for lowest iteration is "+str(Nu*Nv)+","+str(Nu)+"X"+str(Nv))
print("Number of points for highest iteration is "+str(Nu*Nv*max(Elist)**2)+","+str(Nu*max(Elist))+"X"+str(Nv*max(Elist)))

numpoints=0.0
for i in range(0,len(Elist)):
    numpoints=Elist[i]**2.0*Nu*Nv+numpoints
print("Total number of points is "+str(int(numpoints)))

predtime=numpoints/(78000)

print("Runtime: about "+str(format(predtime/60,'.2f'))+" minutes")


r+: 1.31224989991992
r-: 0.6877501000800801
rc: 0.0
Number of points for highest iteration is 106000000.0,53000.0X2000
Total number of points is 106000000
Runtime: about 22.65 minutes


In [5]:
M0=Decimal(M0)
Q=Decimal(Q)
Lambda=Decimal(Lambda)
Tol=Decimal(Tol)
N=Decimal(N)
Nul=Decimal(Nul)
Nup=Decimal(Nup)
#du0=Decimal(du0)
dv0=Decimal(dv0)
scal=int(scal)
scald=Decimal(scal)
umax=Decimal(umax)
vmax=Decimal(vmax)
ru0=Decimal(ru0)
dr0v=Decimal(dr0v)
rplus=Decimal(rplus)
rminus=Decimal(rminus)
rcosm=Decimal(rcosm)
#Nu=int(Nu)
#Nv=int(Nv)

In [6]:
##################################
###Applying Boundary Conditions###

Emax=max([Elist])


rnpf=np.zeros((2,Nv),dtype=datatype)*Decimal(0)
signpf=np.zeros((2,Nv),dtype=datatype)*Decimal(0)
phinpf=np.zeros((2,Nv),dtype=datatype)*Decimal(0)
massnpf=np.full((2,Nv),M0)#np.zeros((2,Nv))
drunpv=np.full((1,Nv-1),-1/(4*dr0v)*(1-2*M0/ru0+Q**2/ru0**2))
dsigunpv=np.zeros((1,Nv),dtype=datatype)


rnpf[0], signpf[0], phinpf[0] = dnu.boundaryv(scal,bdytype,Nv,ru0,dr0v,dv0,vmax,M0,Q,Lambda,scalarfield,datatype)


print(phinpf)
print()


Using Standard Coordinates Along V
[[Decimal('0') Decimal('0') Decimal('0') ..., Decimal('0') Decimal('0')
  Decimal('0')]
 [Decimal('0') Decimal('0') Decimal('0') ..., Decimal('0') Decimal('0')
  Decimal('0')]]



In [7]:
drunptemp=np.empty((Nv))*np.nan
#dsigunp=np.empty((Nu))*np.nan


A=.115
u1=9.5
u2=10

###Applying Propagation Algorithm###

urange=np.array([Decimal(0)],dtype=datatype)
drunp=np.array([Decimal(0)],dtype=datatype)
dsigunp=np.array([Decimal(0)],dtype=datatype)
dphinpuf=np.array([Decimal(0)],dtype=datatype)
massnpf[0][0]=Decimal(1.0)


drunp[0]=-signpf[0][0].exp()/(Decimal(4.0)*dr0v)*(Decimal(1)-Decimal(2)*M0/ru0+Q**Decimal(2.0)/ru0**Decimal(2.0)-Lambda*ru0**Decimal(2/3))
dsigunpvalue=Decimal(1)
drunptemp=Decimal(1)
    

i=0
dv=dv0
if M0>0:
    du0=M0/N
elif M0==0:
    du0=1/N

du=du0

if bdytype=="edd" or bdytype=="fulledd":
    bdyvalue=1.0
    dsigunp[0]=2*(3*Q**2-3*M0*ru0+ru0**4*Lambda)/(ru0*(-3*Q**2+ru0*(6*M0-3*ru0+ru0**3*Lambda)))*drunp[0]
    #dsigunp[0]=2.0*(M0*rnpf[0]-Q**2.0)/(rnpf[0]*(Q**2.0+rnpf[0]*(-2*M0+rnpf[0])))*drunp[0]
else:
    bdyvalue=Decimal(0.0)
    dsigunp[0]=Decimal(0.0)
 
atom = tables.Float64Atom()

rnpfile = tables.open_file('rnp.h5', mode='w',encoding="utf8") 
rnp = rnpfile.create_earray(rnpfile.root, 'data', atom, (0, Nv))

signpfile = tables.open_file('signp.h5', mode='w',encoding="utf8") 
signp = signpfile.create_earray(signpfile.root, 'data', atom, (0, Nv))
phinpfile = tables.open_file('phinp.h5', mode='w',encoding="utf8") 
phinp = phinpfile.create_earray(phinpfile.root, 'data', atom, (0, Nv))
massnpfile = tables.open_file('massnp.h5', mode='w',encoding="utf8") 
massnp = massnpfile.create_earray(massnpfile.root, 'data', atom, (0, Nv))
    
 
#rtemp=rnpf[0]   
#print(rtemp)
rnp.append(rnpf[0].reshape(1,Nv))
signp.append(signpf[0].reshape(1,Nv))
phinp.append(phinpf[0].reshape(1,Nv))
massnp.append(massnpf[0].reshape(1,Nv))

drunp=np.append(drunp,np.nan)
dsigunp=np.append(dsigunp,np.nan)
dphinpuf=np.append(dphinpuf,np.nan)

TempTol=Decimal(1) 
TempTolv=Decimal(1)
Eres=False
while max(urange)<umax and i<50000:
    
    #rnpf=np.insert(rnpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #phinpf=np.insert(phinpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #signpf=np.insert(signpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #massnpf=np.insert(massnpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #drunp=np.insert(drunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dsigunp=np.insert(dsigunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dphinpuf=np.insert(dphinpuf,(1),np.empty((1))*np.nan).reshape(i+2)
    
    #drunp=np.append(drunp,np.nan)
    #dsigunp=np.append(dsigunp,np.nan)
    #dphinpuf=np.append(dphinpuf,np.nan)
    
    du=du0*Tol*TempTol/max([dsigunpvalue.copy_abs(),drunptemp.copy_abs()])
    #du=du0*Tol*TempTol/max(signpf[0,:]).exp()
    #dui=abs(mth.exp(np.nanmax(signpf[0,:]))/(du0*Tol))
    
    if du<1/Nup*Decimal(1.1):
        du=1/Nup
     
    
    if du>=Decimal(1)/Nul:
        #print('r='+str(rnpf[i][0]))
        if rnpf[1][0]<rplus:
            du=Decimal(1)/(1*Nul)
            #print(rplus)
        else:
            du=Decimal(1)/Nul
    
    
    
    
    #print("%.24f" % urange[i])
    #print("%.24f" % du)
    print(urange[i])
    print(du)
    
    if urange[i]>u1 and urange[i]<u2 and vscalarfield==True:
        u=urange[i]+du
        phinpf[1][0]=A*64*(u-u1)**3.0*(u2-u)**3.0/(u2-u1)**6.0
        dphinpuf[i]=192*A*(u-u1)**2.0*(u-u2)**2.0*(-2*u+u1+u2)/(u1-u2)**6.0 
    else:
        phinpf[1][0]=Decimal(0.0)
        dphinpuf[i+1]=Decimal(0.0)
    
    #print(drunp,dsigunp,dphinpuf)
    
    
    rnpf[1][0]=rnpf[0][0]+du*drunp[i]
    #print(type(drunp[0]),type(dsigunp[0]),type(rnpf[0][0]),type(dphinpuf[0]))
    
    drunp[i+1]=drunp[i]+du*(drunp[i]*dsigunp[i]-rnpf[0][0]*dphinpuf[i]**Decimal(2.0))
    
    
    signpf[1][0]=(signpf[0][0]+du*dsigunp[i])#*(1.0-bdyvalue)+np.log(abs(1.0-2.0*M0/rnpf[1][0]+Q**2.0/rnpf[1][0]**2.0))*bdyvalue
    #dsigunp[i+1]=2.0*(M0*rnpf[1][0]-Q**2.0)/(rnpf[1][0]*(Q**2.0+rnpf[1][0]*(-2*M0+rnpf[1][0])))*drunp[i+1]*bdyvalue
    dsigunp[i+1]=Decimal(0)#2*(3*Q**2-3*M0*rnpf[1][0]+rnpf[1][0]**4*Lambda)/(rnpf[1][0]*(-3*Q**2+rnpf[1][0]*(6*M0-3*rnpf[1][0]+rnpf[1][0]**3*Lambda)))*drunp[i+1]*bdyvalue

    massnpf[1][0]=Decimal(M0)
    
    
    

    
    for j in range(0,Nv-1):
        answer=dnu.x4giveraltd(0,j,du,dv,rnpf,phinpf,signpf,massnpf,M0,Q,Lambda,datatype)
        rnpf[1][j+1]=answer[0]
        phinpf[1][j+1]=answer[1]
        signpf[1][j+1]=answer[2]
        massnpf[1][j+1]=answer[3]
        drunptemp=answer[4]
        dsigunpvalue=answer[5]
        exterm1=answer[6]
        exterm2=answer[7]
        exterm3=answer[8]
    
            
            
    #if any(abs((np.exp(signpf[0][:])-np.exp(signpf[1][:]))/np.exp(signpf[0][:]))>0.01):  
    if float(dsigunpvalue*du)>0.1:
        TempTol=TempTol/Decimal(2)
        Eres=True
    else:
        
        rnp.append(rnpf[1].reshape(1,Nv))
        signp.append(signpf[1].reshape(1,Nv))
        phinp.append(phinpf[1].reshape(1,Nv))
        massnp.append(massnpf[1].reshape(1,Nv))
    
        drunp=np.append(drunp,np.nan)
        dsigunp=np.append(dsigunp,np.nan)
        dphinpuf=np.append(dphinpuf,np.nan)
    
        rnpf[0,:]=rnpf[1,:]
        phinpf[0,:]=phinpf[1,:]
        signpf[0,:]=signpf[1,:]
        massnpf[0,:]=massnpf[1,:]
    
        urange=np.append(urange,urange[i]+du)
        
        if Eres==True:
            TempTol=Decimal(2)*TempTol
        else:
            TempTol=Decimal(1)
        Eres=False
        i+=1
    
    print(TempTol)
    print(exterm1)
    print(exterm2)
    print(exterm3)
    print('---')
    
    
    #drunp=np.insert(drunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dsigunp=np.insert(dsigunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dphinpuf=np.insert(dphinpuf,(1),np.empty((1))*np.nan).reshape(i+2)
    #print(urange)
     
    #print(urange)
  
    
    #du=abs(du0*Tol/np.nanmin(drunptemp))

    
    
    
    
    if rnpf[1][0]<0.0 or du.is_nan():
        break
   
    #dumaxlist=[]
    
    
    #if i>1000:
        #break


0
0.01
1
-4.740421594303547788275458896655451E-7
2.9150206636754848162532152297653E-8
0E-39
---
0.01
0.01
1
-4.766677860172115132834439425547050E-7
2.9325783307444047068836151082465E-8
0E-39
---
0.02
0.01
1
-4.793252590337212079983175481542721E-7
2.9503554620367769339082018834664E-8
0E-39
---
0.03
0.01
1
-4.820151467819385684166451241582577E-7
2.9683560695242333386946871551322E-8
0E-39
---
0.04
0.01
1
-4.847380309258928515074553899392733E-7
2.9865842614107661619979145315452E-8
0E-39
---
0.05
0.01
1
-4.874945068822578157402167768191619E-7
3.0050442449944928513871700472523E-8
0E-39
---
0.06
0.01
1
-4.902851842247093498109471624335123E-7
3.0237403296311763828828955626471E-8
0E-39
---
0.07
0.01
1
-4.931106871025307563066577747780280E-7
3.0426769298037185810895105597102E-8
0E-39
---
0.08
0.01
1
-4.959716546740519070759499301979491E-7
3.0618585683020438715349011844435E-8
0E-39
---
0.09
0.01
1
-4.988687415555361153324593547883990E-7
3.0812898795180015344518951932867E-8
0E-39
---
0.10
0.01
1
-

1
-8.977871824494163798438988283591676E-7
5.8177428042035294246085740684672E-8
0E-39
---
0.84
0.01
1
-9.077593305236371566188231043208912E-7
5.8874690385785253305821110658065E-8
0E-39
---
0.85
0.01
1
-9.179557907178800309728730554866938E-7
5.9588211982404133351539254951192E-8
0E-39
---
0.86
0.01
1
-9.283839748378987848403060239370892E-7
6.0318545058969216061764440695815E-8
0E-39
---
0.87
0.01
1
-9.390516191910579532879451151425943E-7
6.1066266562430458746302903565482E-8
0E-39
---
0.88
0.01
1
-9.499668023613903040102460102163810E-7
6.1831979539064765660562084987729E-8
0E-39
---
0.89
0.01
1
-9.611379641582707937249357294047532E-7
6.2616314606477848640952750516417E-8
0E-39
---
0.90
0.01
1
-9.725739258296958594788190220459548E-7
6.3419931525430312265553659769452E-8
0E-39
---
0.91
0.01
1
-9.842839116392809793453505569529852E-7
6.4243520879422518149266036415774E-8
0E-39
---
0.92
0.01
1
-9.962775719150365237297586348413134E-7
6.5087805870698115950798256180850E-8
0E-39
---
0.93
0.01
1
-0.00000

1
-0.0000045970729034694723263922973271753941
3.41034967046754438873482180946564E-7
0E-39
---
1.618147699490702957599840786333988933962
0.008649422387260853716425362629860073999204
1
-0.0000046329226786955498361539743433997862
3.45025887205640348858956561791208E-7
0E-39
---
1.626797121877963811316266148963849007961
0.008305981679889182787194685161599562502813
1
-0.0000046691843096452847586687830207276765
3.49078664406311801756583509292781E-7
0E-39
---
1.635103103557852994103460834125448570464
0.007975969896982392922161128028263683028247
1
-0.0000047058654457472156297653542152619448
3.53194595559273481901347377911238E-7
0E-39
---
1.643079073454835387025621962153712253492
0.007658882919554211887746585053174996876563
1
-0.0000047429732347764486685746982730983597
3.57374960303514669288439599967572E-7
0E-39
---
1.650737956374389598913368547206887250369
0.007354233160716694151524908152586912828646
1
-0.0000047805149230862425189992520977056506
3.61621065975562944553917932163249E-7
0E-39
---
1

1
-0.0000068033790975445274556743367370649568
6.13650885409014850041689154471534E-7
0E-39
---
1.801591389939434243420046901373627518166
0.001320394611753512132633509986816497798610
1
-0.0000068678651080130734105692820159186235
6.22387835940516081955260033398276E-7
0E-39
---
1.802911784551187755552680411360444015965
0.001267169252206433610158042339053101870927
1
-0.0000069332731833347198648746401758385804
6.31291506146250154967745482344123E-7
0E-39
---
1.804178953803394189162838453699497117836
0.001216073241406482313918203829666922795674
1
-0.0000069996209993536086660389438182983354
6.40365893095124372565497971820995E-7
0E-39
---
1.805395027044800671476756657529164040632
0.001167021825548136459801229162735852708060
1
-0.0000070669266554639578578790495741429028
6.49615109221133735298079674874748E-7
0E-39
---
1.806562048870348807936557886691899893340
0.001119933612847863734552650365778797827295
1
-0.0000071352086870506546001539775758461998
6.59043386241593903323898472582789E-7
0E-39
---
1

1
-0.0000109515254276785845123100318742527100
0.000001252190069778001744813436576451891
0E-39
---
1.829127968859458614783202054397442392893
0.0002039667469423656248121508410159613878418
1
-0.0000110835486270971832012472601085824144
0.000001274867049595842630973379632078958
0E-39
---
1.829331935606400980408014205238458354281
0.0001955849462068500095111191496542062598333
1
-0.0000112179748928761664117401967169839631
0.000001298094985039813862752457042119266
0E-39
---
1.829527520552607830417525324388112560541
0.0001875430953096470162621187444107738210473
1
-0.0000113548628915331986664167782583352268
0.000001321890706238120016065288565050721
0E-39
---
1.829715063647917477433787443132523334362
0.0001798275406260401564529968462150496242644
1
-0.0000114942730905032740562403874896514691
0.000001346271663775587288407769822533782
0E-39
---
1.829894891188543517590240439978738383986
0.0001724251730029713688869246446626505686844
1
-0.0000116362678248464845765757336166728753
0.0000013712559555267807

...

1
1.762543088433065942E-21
-1.58950791440451182986E-20
0E-39
---
1.933710456742101732018603753147498548735
0.002122017884373763845681308124892396232668
1
1.632069377882849874E-21
-1.47184327119701755718E-20
0E-39
---
1.935832474626475495864285061272390944968
0.002167072144576934383980020716929398863830
1
1.511250191528258094E-21
-1.36288533347951266090E-20
0E-39
---
1.937999546771052430248265081989320343832
0.002213077634157459495933389761880452762713
1
1.399371313114776625E-21
-1.26199000211720566665E-20
0E-39
---
1.940212624405209889744198471751200796595
0.002260054155581371285429046646807120316898
1
1.295771353556291721E-21
-1.16856081891544046297E-20
0E-39
---
1.942472678560791261029627518398007916912
0.002308021908032327099534452421897016072598
1
1.199837843996220385E-21
-1.08204544323868121400E-20
0E-39
---
1.944780700468823588129161970819904932985
0.002357001494448072815336372073851343618291
1
1.111003617791746318E-21
-1.00193238918801232723E-20
0E-39
---
1.947137701963271660944

1
2.5461078253814077E-23
-2.296146160235633867E-22
0E-39
---
2.150625786131630903353271180830783364612
0.006692330426476886135586276006737130660682
1
2.3567104537803126E-23
-2.125342646403413555E-22
0E-39
---
2.157318116558107789488857456837520495273
0.006831909946562073384681911598785657927189
1
2.1813660314826373E-23
-1.967212452274578418E-22
0E-39
---
2.164150026504669862873539368436306153200
0.006974292653974419259397959841517672920537
1
2.0190332493666673E-23
-1.820816497796266689E-22
0E-39
---
2.171124319158644282132937328277823826121
0.007119528257362539030123973416923886821973
1
1.8687478997548596E-23
-1.685285235036213403E-22
0E-39
---
2.178243847416006821163061301694747712943
0.007267666914682248714337315393806471112063
1
1.7296171702456561E-23
-1.559813502209290700E-22
0E-39
---
2.185511514330689069877398617088554184055
0.007418759204454893618607261508980181868638
1
1.6008143597257517E-23
-1.443655758437559584E-22
0E-39
---
2.192930273535143963496005878597534365924
0.0075728

1
1.65468405423064E-25
-1.4922663003079146E-24
0E-39
---
2.824434558818230971969874739078697485001
0.01
1
1.57417630922405E-25
-1.4196612824516571E-24
0E-39
---
2.834434558818230971969874739078697485001
0.01
1
1.49823953310767E-25
-1.3511785186027875E-24
0E-39
---
2.844434558818230971969874739078697485001
0.01
1
1.42657491702240E-25
-1.2865485344111418E-24
0E-39
---
2.854434558818230971969874739078697485001
0.01
1
1.35890577603334E-25
-1.2255218074019228E-24
0E-39
---
2.864434558818230971969874739078697485001
0.01
1
1.29497572055984E-25
-1.1678671179364963E-24
0E-39
---
2.874434558818230971969874739078697485001
0.01
1
1.23454699449014E-25
-1.1133700504918749E-24
0E-39
---
2.884434558818230971969874739078697485001
0.01
1
1.17739896338982E-25
-1.0618316302971630E-24
0E-39
---
2.894434558818230971969874739078697485001
0.01
1
1.12332673800142E-25
-1.0130670819777558E-24
0E-39
---
2.904434558818230971969874739078697485001
0.01
1
1.07213991981357E-25
-9.669046982837521E-25
0E-39
---
2.914434

1
7.014175612075E-27
-6.32575157173200E-26
0E-39
---
3.634434558818230971969874739078697485001
0.01
1
6.789225347256E-27
-6.12287982823986E-26
0E-39
---
3.644434558818230971969874739078697485001
0.01
1
6.571967015411E-27
-5.92694505208883E-26
0E-39
---
3.654434558818230971969874739078697485001
0.01
1
6.362108179508E-27
-5.73768350969452E-26
0E-39
---
3.664434558818230971969874739078697485001
0.01
1
6.159368862881E-27
-5.55484270478729E-26
0E-39
---
3.674434558818230971969874739078697485001
0.01
1
5.963480958703E-27
-5.37818084585424E-26
0E-39
---
3.684434558818230971969874739078697485001
0.01
1
5.774187670335E-27
-5.20746634141833E-26
0E-39
---
3.694434558818230971969874739078697485001
0.01
1
5.591242980772E-27
-5.04247732156398E-26
0E-39
---
3.704434558818230971969874739078697485001
0.01
1
5.414411149541E-27
-4.88300118421695E-26
0E-39
---
3.714434558818230971969874739078697485001
0.01
1
5.243466235490E-27
-4.72883416477868E-26
0E-39
---
3.724434558818230971969874739078697485001
0.01


1
4.81061100030E-28
-4.3384363917017E-27
0E-39
---
4.464434558818230971969874739078697485001
0.01
1
4.64511043808E-28
-4.1891788196925E-27
0E-39
---
4.474434558818230971969874739078697485001
0.01
1
4.48464484175E-28
-4.0444620602474E-27
0E-39
---
4.484434558818230971969874739078697485001
0.01
1
4.32907339529E-28
-3.9041591186900E-27
0E-39
---
4.494434558818230971969874739078697485001
0.01
1
4.17825944698E-28
-3.7681467559080E-27
0E-39
---
4.504434558818230971969874739078697485001
0.01
1
4.03207035843E-28
-3.6363053522129E-27
0E-39
---
4.514434558818230971969874739078697485001
0.01
1
3.89037735937E-28
-3.5085187763718E-27
0E-39
---
4.524434558818230971969874739078697485001
0.01
1
3.75305540800E-28
-3.3846742596546E-27
0E-39
---
4.534434558818230971969874739078697485001
0.01
1
3.61998305671E-28
-3.2646622747472E-27
0E-39
---
4.544434558818230971969874739078697485001
0.01
1
3.49104232307E-28
-3.1483764193857E-27
0E-39
---
4.554434558818230971969874739078697485001
0.01
1
3.36611856588E-28


In [8]:
rnpfile.close()
signpfile.close()
phinpfile.close()
massnpfile.close()

In [9]:
Nu=i+2

print(Nu)
#rnpf[i+1][j+1]=answer[0]
#phinpf[i+1][j+1]=answer[1]
#signpf[i+1][j+1]=answer[2]
#massnpf[i+1][j+1]=answer[3]
#drunp[i+1][j+1]=answer[4]
#dsigunp[i+1][j+1]=answer[5]

1295


In [10]:
#print(rnpf[1,0])

In [11]:
#print(dnu.x4giveralt(0,0,du,dv,rnpf,phinpf,signpf,Q,Lambda))

In [12]:
#print(rnpf)

In [13]:
vrange=np.arange(0,float(vmax),float(dv0))
urange=urange.astype(float)
#print(urange)
print(urange)

[ 0.          0.01        0.02       ...,  5.28443456  5.29443456
  5.30443456]


In [14]:
np.save('rnputil',np.array([float(Nu*Nv),float(Nu),float(Nv),float(ru0),float(dr0v),float(M0),float(Q),float(Lambda),wHD]))
np.save('urange',urange)
np.save('vrange',vrange)

In [15]:
stop = timeit.default_timer()

In [16]:
acttime=stop - start

numpoints=Nu*Nv



print(str(acttime)+' seconds')
print(str((acttime)/60)+' minutes')
print(str((acttime)/3600)+' hours')

print(str(numpoints)+' points')

print(str((numpoints)/(stop - start))+' points per second') 

exttimef=(acttime-predtime)/60

exttime=format(abs((acttime-predtime)/60),'.2f')
if exttimef>0.0:
    print("Took "+str(exttime)+" more minutes")
if exttimef<0.0:
    print("Took "+str(exttime)+" less minutes")
if exttimef==0.0:
    print("Took exactly the right time!!!!")

531.1315837722311 seconds
8.85219306287052 minutes
0.14753655104784197 hours
2590000 points
4876.38106852009 points per second
Took 13.80 less minutes


conv=False
j=0
tempanswer=[]
    while conv==False:
        dv=TempTolv*dv0
        jcount=1
        while j<Nv-1:
            j1=mth.ceil(jcount*TempTolv)
            j2=mth.floor(jcount*TempTolv)
            answer=dnu.x4giveralt(0,j+mth.ceil((jcount-1)*TempTolv),du,dv*TempTolv,rnpf,phinpf,signpf,massnpf,M0,Q,Lambda,datatype)
            rnpf[1][j+j1]=answer[0]
            phinpf[1][j+j1]=answer[1]
            signpf[1][j+j1]=answer[2]
            massnpf[1][j+j1]=answer[3]
            #drunptemp[j+1]=answer[4]
            dsigunpv=answer[5]
            jcount=(jcount+1)**(0**j2)
            j=j+j2
        
        if np.abs(tempanswer[0]-answer[0])/answer[0]>0.01 or not tempanswer:
            tempanswer=answer
            TempTol=TempTolv/2
            
        else:
            conv=True
            

In [17]:
print(dsigunpv)

[[0 0 0 ..., 0 0 0]]
